# goal strukture 

## phase one: 
>track_name, track_artist, id, lyrics (empty)

## phase two: 
>track_name, track_artist, lyrics, id

## phase three: 
>track_name, track_artist, lyrics, embedding, id


# Phase one preprocessing

In [2]:
import pandas as pd 
import numpy as np

# importing the raw data sets 

df_MSSS = pd.read_csv("../raw_data/Most_Streamed_Spotify_Songs_2024.csv", encoding='latin1')
df_SMSD = pd.read_csv("../raw_data/spotify_millsongdata.csv")
df_SS = pd.read_csv("../raw_data/spotify_songs.csv")

# cleaning them to only the two rows we care about 

df_MSSS_reduced = df_MSSS[["Artist", "Track"]].rename(columns={"Artist": "track_artist", "Track": "track_name"})
df_MSSS_reduced["track_lyrics"] = None

df_SMSD_reduced = df_SMSD[["artist", "song", "text"]].rename(columns={"artist": "track_artist", "song": "track_name", "text": "track_lyrics"}) # also has the text so here we are goingt to put the text from the stuff here already
df_SS_reduced = df_SS[["track_name", "track_artist"]]
df_SS_reduced["track_lyrics"] = None


# combine dfs 

df_base = df_MSSS_reduced
df_base = pd.concat([df_base, df_SMSD_reduced], ignore_index=True)
df_base = pd.concat([df_base, df_SS_reduced], ignore_index=True)

# del duplicates 
df_clean = df_base.drop_duplicates(subset=['track_artist', 'track_name'])

# adding ids

import uuid

def generate_uuid():
    return str(uuid.uuid4())

df_clean['track_id'] = [generate_uuid() for _ in range(len(df_clean))]

df = df_clean


# about 86000 rows 

C:\Users\lipka\AppData\Local\Temp\ipykernel_9640\1460722427.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_SS_reduced["track_lyrics"] = None
C:\Users\lipka\AppData\Local\Temp\ipykernel_9640\1460722427.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['track_id'] = [generate_uuid() for _ in range(len(df_clean))]


# Phase two preprocessing

In [12]:
from dotenv import load_dotenv
import requests
import os
from bs4 import BeautifulSoup
import pandas as pd

load_dotenv()

df = pd.read_csv("big_song_data.csv")

print(df) # need to remove the wierd index thing to iterrows again 


API_TOKEN = os.getenv('client_access_token')
BASE_URL = 'https://api.genius.com'


def search_song(artist: str, title: str):
    search_url = f"{BASE_URL}/search"
    headers = {'Authorization': f'Bearer {API_TOKEN}'}
    params = {'q': f"{title} {artist}"}
    response = requests.get(search_url, headers=headers, params=params)
    return response.json()

# Function to get song lyrics
def get_lyrics(song_api_path: str):
    song_url = f"{BASE_URL}{song_api_path}"
    headers = {'Authorization': f'Bearer {API_TOKEN}'}
    response = requests.get(song_url, headers=headers)
    json_response = response.json()
    path = json_response['response']['song']['path']
    
    # Fetch the song lyrics from the Genius website (not directly available via API)
    page_url = f"https://genius.com{path}"
    page = requests.get(page_url)
    soup = BeautifulSoup(page.text, 'html.parser')

    # Find the lyrics container with the data-lyrics-container="true" attribute
    lyrics_div = soup.find('div', {'data-lyrics-container': 'true'})
    lyrics = lyrics_div.get_text(separator='\n') if lyrics_div else "Lyrics not found."
    return lyrics

def get_lyrics_for_song(artist: str, title: str, iteration: int) -> str:
    search_result = search_song(artist, title)
    if search_result['response']['hits']:
        song_api_path = search_result['response']['hits'][0]['result']['api_path']
        print(f"{iteration} - FOUND - Title: {title} - Artist: {artist}")
        return get_lyrics(song_api_path)            
    print(f"{iteration} - FAIL - Title: {title} - Artist: {artist}")
    return None

# iterating through the df in batches - something like 500 songs 
# throwing it at threads because its a task that can be perfectly threaded
# saving the batches to save progress 

for idx, row in df.iterrows():
    if row['track_lyrics'] != None:
        continue

    artist = row["track_artist"]
    track_name = row["track_name"]

    df.at[idx, 'track_lyrics'] = get_lyrics_for_song(artist, track_name, idx)
    df.to_csv("big_song_data.csv", index=False)



       Unnamed: 0    track_artist                            track_name  \
0               0   Tommy Richman                   MILLION DOLLAR BABY   
1               1  Kendrick Lamar                           Not Like Us   
2               2         Artemas            i like the way you kiss me   
3               3     Miley Cyrus                               Flowers   
4               4          Eminem                               Houdini   
...           ...             ...                                   ...   
86714       86714    Lush & Simon  City Of Lights - Official Radio Edit   
86715       86715  Tegan and Sara   Closer - Sultan & Ned Shepard Remix   
86716       86716     Starkillers          Sweet Surrender - Radio Edit   
86717       86717          Mat Zo        Only For You - Maor Levi Remix   
86718       86718    Julian Calor                Typhoon - Original Mix   

                                            track_lyrics  \
0      [Intro]\nDo it, baby, do what I 